In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import datetime
import uule_grabber
import time
import secrets


chromedriver_autoinstaller.install()
dt_csv = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
filegeo = 'geo_u.txt'
with open(f'{filegeo}', 'r', encoding="utf-8") as f:
        proxies1 = list(filter(lambda x: len(x.strip()) > 0, f))
        geo = list(map(lambda x: x.replace('\n', ''), proxies1))
        
filegeo = 'keywords_kenzi.txt'
with open(f'{filegeo}', 'r', encoding="utf-8") as f:
        proxies1 = list(filter(lambda x: len(x.strip()) > 0, f))
        keywords = list(map(lambda x: x.replace('\n', ''), proxies1))

hl = 'ru' #hl=sv          Swedish https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes
country = 'ua' #se 	Sweden  
num = '10' # Кількість результатів пошуку
zone = 'com.ua' #'com.ua'
# region = 'Kyiv,Kyiv city,Ukraine'  #'Kyiv,Kyiv city,Ukraine', 'UA'   New York,New York,United States
# uule = uule_grabber.uule(region)

for region in geo:
    uule = uule_grabber.uule(region)
    city = region.split(',')[0]
    keyword = secrets.choice(keywords)

    chrome_options = webdriver.ChromeOptions()
    chrome_options.headless = True
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("start-maximized")
#         mobile_emulation = { "deviceName": "Nexus 5", "userAgent": "Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexus 5 Build/JOP40D) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.166 Mobile Safari/535.19" } }

#         chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)

    dt = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")

    print(city, keyword)
    browser = webdriver.Chrome(options=chrome_options)
    browser.get(f'https://google.com.ua/search?q={keyword}&pws=0&uule={uule}&hl={hl}&gl={country}')
    time.sleep(2)
    
    opys = {}
    brands = {}
    try:
        if browser.find_elements(By.XPATH, "//div[@id='result-stats']"):
            browser.save_screenshot(f'screenshots/{dt}_{city}_desk.png')
            brend = browser.find_elements(By.XPATH, "//div[@class='LbUacb']")
            brends = list(map(lambda x: x.text, brend))
            brands[keyword] = brends
#             print(brends)
            title = browser.find_elements(By.XPATH, "//span[@class='pymv4e']")
            titles = list(map(lambda x: x.text, title))
#             print(titles)
#             opys[keyword] = [brends, titles]
            opys[keyword] = [list(tup) for tup in zip(brends, titles)]
            file = open(f"{dt_csv}_vydacha.csv", "a", encoding="utf-8")

            # make write object
            writer = csv.writer(file, lineterminator = '\n')
            for key, value in opys.items():
                writer.writerow([key, value])
            file.close()
            file = open(f"{dt_csv}_brands.csv", "a", encoding="utf-8")

            # make write object
            writer = csv.writer(file, lineterminator = '\n')
            for key, value in opys.items():
                writer.writerow([key, value])
            file.close()

            
    except:pass
#     break

    browser.close()
    browser.quit()
    time.sleep(15)   


Vinnytsia чехол книжка айфон 12 про макс
['Nillkincase.com...', 'ВЧехле', 'WOMUX', 'KENZI', 'Nillkincase.com...', 'chekhol.com.ua', 'bestcover.com.ua', 'Mobile-Case', 'Іван Чохол', 'WOMUX', 'AKS.ua', 'VEST', 'ITFridge', 'ВЧехле', 'KENZI', 'Nillkincase.com...', 'iLounge', 'WOMUX', 'Zakazuy', 'Top-best.ua', 'WOMUX', 'ВЧехле', 'Kiber Panda', 'Saltov', 'Nillkincase.com...', 'mocolo.org.ua', 'WOMUX', 'chekhol.com.ua', 'WOMUX']
['Чохол-книжка Dux Ducis з кишенею для візиток для Apple iPhone 12 Pro Max (6.7 "), Черный', '', 'Кожаный чехол (книжка) Nillkin Qin Pro Camshield для Apple iPhone 13 Pro Max (6.7")', '', 'Чехол книжка из натуральной кожи магнитный противоударный для IPhone 12 Pro "BOTTEGA" Коричневый - No1', '', 'Чехол книжка для iPhone 12 Pro Max - Flip Magnetic Case (чёрный)', '', 'Чохол-книжка Dux Ducis з кишенею для візиток для Apple iPhone 12 Pro Max (6.7 "), Синий', '', 'Кожаный чехол-книжка QIALINO Classic Case для iPhone 14 Pro Max - коричневый', '', 'Чехол-книжка Litchie Wal

Kryvyi Rih чехол книжка iphone 11 pro max
['Zakazuy', 'ВЧехле', 'WOMUX', 'Zakazuy', 'iLounge', 'KENZI', 'Іван Чохол', 'Zakazuy', 'AKS.ua', 'Zakazuy', 'VEST', 'Mobile-Case', 'Top-best.ua', 'ВЧехле', 'iLounge', 'WOMUX', 'shock.org.ua', 'Zakazuy', 'Іван Чохол', 'y.ua', 'Саме тут!', 'ВЧехле', 'WOMUX', 'e-star.ua']
['Чехол-книжка с натуральной кожи Pierre Cardin Wallet для Iphone 11 Pro Max - Черный', '', 'Кожаный чехол (книжка) Nillkin Qin Pro Camshield для Xiaomi Redmi Note 11 Pro 5G', '', 'Чехол книжка противоударный магнитный КОЖАНЫЙ влагостойкий для Iphone 11 Pro MAX "VERSANO" No8 - Черный', '', 'Чехол-книжка с натуральной кожи Pierre Cardin Wallet для Iphone 11 Pro Max - Темно-коричневый', '', 'Шкіряний чохол-гаманець iLoungeMax Leather Folio Midnight Blue для iPhone 11 Pro OEM, Лучшая копия Apple - Синій, Копії Apple', '', 'Чехол книжка для iPhone 11 Pro Max - Flip Magnetic Case (марсала)', '', 'Чехол Apple Leather Folio для iPhone 11 Pro Max Black Original (MX082)', '', 'Чехол-книжк

Zaporizhzhia чехол книжка для xiaomi mi 9 se
['ROZETKA', 'e-star.ua', 'Labaks', 'Top-best.ua', '0629store.com....', 'magazin-shop', 'ВЧехле', 'Mobile-Case', 'Steklon.com.ua', 'bestcover.com.ua', 'Oneaccs', 'Zorrov', 'skl.com.ua', 'WOMUX', 'cherdak-store.c...', 'Eldorado', 'Top-best.ua', 'ROZETKA', 'Cherdak-store', 'Top-best.ua', 'Oneaccs', '0629store.com....', 'Labaks', 'ROZETKA', 'WOMUX', 'Mobile-Case', 'kili', 'Kiber Panda', 'ВЧехле']
['Чохол Idewei для Xiaomi Mi 9 SE книжка шкіра PU чорний. Чохли для мобільних телефонів', '', 'Чехол книжка для Xiaomi Mi9 SE Mofi Crystal Black (Черный)', '', 'Чехол книжка Premium кожаный Xiaomi Mi 9 SE Черный', '', 'Чехол книжка для Xiaomi Mi9 SE Mofi Vintage Blue (Синий)', '', 'Чехол GC для Xiaomi Mi 9 SE книжка магнитная Grey', '', 'Чехол книжка Idewei для Xiaomi Mi 9 SE с застежкой', '', 'Чехол Nillkin Matte для Xiaomi Mi 9', '', 'Чехол-книжка Litchie Wallet для Xiaomi Mi 9 SE Белый', '', 'Чехол книжка кожанный Xiaomi Mi 9 SE (Черный)', '', 'Чехол

Kropyvnytskyi samsung galaxy j7 чехол книжка
['ВЧехле', 'KENZI', 'Eldorado', 'ROZETKA', 'ITFridge', 'magazin-shop', 'AKS.ua', 'Labaks', 'ROZETKA', 'Prom', 'KENZI', 'ROZETKA', 'Oneaccs', 'Zorrov', 'WOMUX', 'Cheholl.com', 'Vseplus.com', 'ROZETKA', 'ITFridge', 'Eldorado', 'ROZETKA', 'ROZETKA', 'ITFridge', 'Prom']
['Кожаный чехол (книжка) Classy для Samsung J710F Galaxy J7 (2016)', '', 'Чехол книжка для Samsung J7 (2017) - Flip Magnetic Case (чёрный). Защитный магнитный чехол для телефона', '', 'Чохол-книжка GELIUS FB Leather для Samsung Galaxy A032 (A03 Core) Black (89746)', '', 'Чехол книжка Fashion Case для Samsung Galaxy J7 2015 (j700) Черный. Чехлы для мобильных телефонов', '', 'Чехол книжка Level для Samsung Galaxy J7 2017 J730 Экокожа Зеленый', '', 'Чехол книжка Magnei для Samsung Galaxy J7 Neo (J701), J7 2015 (J700) с магнитом', '', 'Чехол Level Samsung J730 Galaxy J7 2017 Black', '', 'Чехол книжка Premium кожаный Samsung Galaxy J7 (2017) Черный', '', 'Чохол-книжка ColorWay Elegant

Odessa чехол книжка на xiaomi redmi note 7
['ВЧехле', 'ROZETKA', 'ITFridge', 'KENZI', 'Zorrov', 'ВЧехле', 'WOMUX', 'ITFridge', 'magazin-shop', '4you', 'ВЧехле', 'oneklik', 'Zakazuy', 'Prom', 'itsell', 'ROZETKA', 'Labaks', 'Oneaccs', 'Cherdak-store', 'WOMUX', 'ROZETKA', 'АЛЛО', 'ВЧехле', 'Посуд Шеф']
['Шкіряний чохол (книжка) Classy для Xiaomi Redmi Note 7', '', 'Чохол-книжка ColorWay Elegant Book для Xiaomi Poco M4 Pro 5G Black (CW-CEBXPM4P5-BK). Чохли для мобільних телефонів', '', 'Чехол книжка Level для Xiaomi Redmi Note 7 Экокожа Бордовый', '', 'Чехол книжка для Xiaomi Redmi Note 7 - Flip Magnetic Case (марсала)', '', 'Чехол для Xiaomi Redmi Note 7 / 7Pro черный матовый soft touch', '', 'Кожаный чехол (книжка) Art Case с визитницей для Xiaomi Redmi Note 7 Pro', '', 'Чехол книжка противоударный магнитный КОЖАНЫЙ влагостойкий для XIAOMI Redmi NOTE 7 / 7 pro "GOLDAX"', '', 'Чехол книжка Level для Xiaomi Redmi Note 7 Экокожа Синий', '', 'Чехол книжка Magnei для Xiaomi Redmi 7 с магнитом

Ternopil чехол книжка honor 9x
['KENZI', 'Labaks', '0629store.com....', 'VEST', 'ROZETKA', 'skl.com.ua', 'KENZI', 'WOMUX', 'Steklon.com.ua', 'CityTorg', 'Prom', 'Zorrov', 'ROZETKA', 'VEST', 'АЛЛО', 'ROZETKA', 'Eldorado', 'WOMUX', 'IZI.UA', 'vst-shop.com.ua', 'Prom', 'mocolo.org.ua', 'e-star.ua', 'VEST']
['Чехол книжка для Honor 9X - Flip Magnetic Case (чёрный). Защитный магнитный чехол для телефона', '', 'Чехол книжка Premium кожаный Huawei Honor 9X Черный', '', 'Чехол G-Case для Honor 9X China книжка Ranger Series магнитная Black', '', 'Чехол-книжка зеркало Epik Clear View для Huawei Honor 9X (China) фиолетовый', '', 'Чохол G-Case для Honor 9X China книжка Ranger Series магнітна Black. Чохли для мобільних телефонів', '', 'Чехол-книжка Huawei Honor 9X – Fashion', '', 'Чехол книжка для Honor 9X - Flip Magnetic Case (золотой)', '', 'Чехол книжка противоударный магнитный КОЖАНЫЙ влагостойкий для Honor 9X Lite "VERSANO" No8 - Черный', '', 'Чехол книжка кожаный Huawei Honor 9X (Черный)', ''

Chernihiv чехол книжка самсунг а50
['MobiFun Store', 'KENZI', 'ITFridge', 'ВЧехле', 'Oneaccs', 'shock.org.ua', 'skl.com.ua', 'ВЧехле', 'WOMUX', 'Labaks', 'Zakazuy', 'ВЧехле', 'bestcover.com.ua', 'ROZETKA', 'chekhol.com.ua', 'mocolo.org.ua', 'Oneaccs', 'Kiber Panda', 'KENZI', 'ВЧехле', 'WOMUX', 'Zorrov', 'Oneaccs', 'Oneaccs']
['Чехол-книжка Samsung Galaxy A50 (2019) Бронза', '', 'Чехол книжка для Samsung A50 - Flip Magnetic Case (чёрный). Защитный магнитный чехол для телефона', '', 'Чехол книжка Level для Samsung Galaxy A50 A505 Экокожа Черный', '', 'Кожаный чехол (книжка) Art Case с визитницей для Samsung Galaxy A50 (A505F)', '', 'Чехол книга для Samsung Galaxy A50 с эко кожи с подставкой магнитом книжка на телефон самсунг а50 синяя STN', '', 'Чехол-книжка Nillkin Sparkle Leather Case Samsung Galaxy A50 Black', '', 'Чехол-книжка Samsung Samsung Galaxy A50 – Fashion', '', 'Чехол-книжка Dux Ducis с карманом для Samsung Galaxy A50 (A505F)', '', 'Чехол книжка противоударный магнитный КОЖАН

In [43]:
l1 = [1,2,3,4]

l2 = [11,22,33,44]

[list(tup) for tup in zip(l1, l2)]

# In [27]: [list(tup) for tup in zip(l1, l2)]
# Out[27]: [[1, 11], [2, 22], [3, 33], [4, 44]]

[[1, 11], [2, 22], [3, 33], [4, 44]]

In [42]:
f

In [35]:
brends = browser.find_elements(By.XPATH, "//div[@class='LbUacb']")
titles = list(map(lambda x: x.text, brends))
print(titles)
# for j in brends:
#     try:
#         print(j.text)
#     except:pass

['KENZI', 'Labaks', 'magazin-shop', 'Mobile-Case', 'Vseplus.com', 'bestcover.com.ua', 'ROZETKA', 'Zorrov', 'KENZI', 'e-star.ua', 'ВЧехле', 'Easybay']


In [21]:
brends1 = brends[0].find_element(By.XPATH, "//span[@class='zPEcBd LnPkof']")

In [25]:
brends[.find_element(By.CLASS_NAME, "zPEcBd LnPkof").get_attribute("textContent")

<selenium.webdriver.remote.webelement.WebElement (session="6845ae67dff1713d4641968c9387cf77", element="305fcd56-d7b0-489a-a0bd-1f4a3eb5234f")>

In [36]:
import csv
csv_columns = ['No','Name','Country']
dict_data = [
{'No': 1, 'Name': 'Alex', 'Country': 'India'},
{'No': 2, 'Name': 'Ben', 'Country': 'USA'},
{'No': 3, 'Name': 'Shri Ram', 'Country': 'India'},
{'No': 4, 'Name': 'Smith', 'Country': 'USA'},
{'No': 5, 'Name': 'Yuva Raj', 'Country': 'India'},
]
csv_file = "Names.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")

In [38]:
import csv

dict = {"Name": "Vishvajit Rao", "age": 23, "Occupation": "Developer","Skills": "Python"}
file = open("sample.csv", "w")

# make write object
writer = csv.writer(file, lineterminator = '\n')

for key, value in dict.items():
    writer.writerow([key, value])

file.close()